In [1]:
import sys

print(sys.version)

3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]


#   클래스 내부의 메소드 처리하기

- 보통 객체는 행위 즉 메소드를 가지고 행동을 수행한다.
- 메소드는 인스턴스 메소드, 클래스 메소드 정적 메소드가 있다.
- 속성을 보호하고 이 속성을 접근하는 인스턴스 메소드를 프로퍼티로 처리한다.

# 1  property 사용법

- 속성을 참조할 때 게터/세터/딜리터 메소드를 사용해서 참조한다.
- 프로퍼티 처리는 클래스 내의 메소드를 데코레이터로 묶는다.



## 클래스내의 조회 프로퍼티 정의 
- 하나의 속성을 가지고 있다. 
- 이 속성을 조회하는 하나의 메소드를 프로퍼티로 연계한다. 

In [1]:
class A:
    def __init__(self, value):
        self._double = value
        
    @property
    def double(self):
        return self._double * 2

In [2]:
a = A(10)

In [3]:
a._double, a.double

(10, 20)

## 클래스를 정의할 때 객체의 속성을 조회 및 갱신 프로퍼티 처리

- 프로퍼티 이름으로 접근하므로 실제 속성의 이름에 _ 붙여서 변경한다
- 조회하는 첫번째 프로퍼티가 data로 생성된다
- 갱신과 삭제는 data.setter, data.getter로 데코레이터를 처리한다.
- 이 클래스에는 초기화 메소드가 없지만 setter에서 속성을 추가한다.

In [4]:
class OnlyInt:

    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        if isinstance(value, int):
            self._data = value
        else:
            raise AttributeError("only int")
            
    @data.deleter
    def data(self):
        raise AttributeError("can't")

In [5]:
t = OnlyInt()

In [6]:
t.__dict__

{}

## 프로퍼티를 지정하지만 객체 속성은 추가 삭제 가능

### 객체의 속성 추가
- 파이썬은 객체의 속성을 언제라도 추가할 수 있다.

In [7]:
t.data = 12

In [8]:
t.data

12

### 객체의 속성 삭제

In [9]:
try :
    del t.data
except Exception as e :
    print(e)

can't


### 재할당을 하면 추가된다. 

In [10]:
try :
    t.data = '123'
except Exception as e :
    print(e)

only int


##  프로퍼티를 특정 계산식 메소드로 처리하기

- 속성에 대한 처리대신 특정 계산을 조회하는 프로퍼티로 처리가 가능하다.


In [11]:
class Circle:
    pi = 3.14
    
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def diameter(self):
        return self.radius * 2

    @property
    def circumference(self):
        return self.radius * self.pi * 2
    
    @property
    def area(self):
        return self.radius ** 2 * self.pi

#     area = area.setter(no)
#     circumference = circumference.setter(no)

In [12]:
c = Circle(10)

In [13]:
try :
    c.area = 12
except Exception as e :
    print(e)

can't set attribute


## 2 사용자 정의 property 구현체

- 프로퍼티는 3개의 함수를 저장하는 속성이 있다.
- 데코레이터를 처리할 때 3개의 메소드에서 함수를 내부에 저장한다.
- 이름으로 부르면 각각의 저장된 함수가 메소드로 처리된다.


In [14]:
class my_property:
    def __init__(self, fget=None, fset=None, fdel=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        
    def setter(self, fset):
        self.fset = fset
        return self 
    
    def deleter(self, fdel):
        self.fdel = fdel
        return self
    
    def getter(self, fget):
        self.fget = fget
        return self

    def __get__(self, instance, owner):
        return self.fget(instance)
    
    def __set__(self, instance, value):
        if not self.fset:
            raise AttributeError
        return self.fset(instance, value)
    
    def __delete__(self, instance):
        if not self.fdel:
            raise AttributeError
        return self.fdel(instance)

### 사용자 정의 프로퍼티 처리하기

In [15]:
class OnlyInt:
    @my_property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        if isinstance(value, int):
            self._data = value
        else:
            raise AttributeError("only int")
            
    @data.deleter
    def data(self):
        raise AttributeError("can't")

In [16]:
c = OnlyInt()

In [17]:
c.data = 12

In [18]:
c.data

12

In [19]:
try :
    c.data = '123'
except Exception as e :
    print(e)

only int


#  3.   classmethod
- 클래스가 호출해서 사용할 수 있는 메소드
- 모든 객체도 이 클래스 메소드를 같이 사용할 수 있다.
- 객체들이 동일한 처리가 필요할 경우 클래스 메소드로 처리한다.


In [20]:
class A:
    def m(arg):
        print(arg)

    @classmethod
    def cm(arg):
        print(arg)

In [21]:
a = A()

In [22]:
a.m()

In [23]:
a.cm()

<class '__main__.A'>


## 도전 : classmethod 구현

In [24]:
from functools import partial


class my_classmethod:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, owner):
        return partial(self.func, owner)

### 인스턴스 메소드와 클래스 메소드 첫번째 자리 확인

- 객체와 클래스 들어온 것을 확인한다.
- self와 cls는 일반 변수명이다.


In [25]:
class A:
    def m(arg):
        print(arg)

    @my_classmethod
    def cm(arg):
        print(arg)

In [26]:
a = A()

In [27]:
a.m()

In [28]:
a.cm()

<class '__main__.A'>


# 4.  staticmethod

- 클래스나 객체에서 함수를 그대로 사용할 수 있도록 한다.
- 이 메소드로 데코레이터를 처리하면 첫번째 자리에 객체나 클래스가 자동으로 세팅되지 않아서 정의한 매개변수에 인자를 전부 세팅해야한다.

In [29]:
class A:
    @staticmethod
    def sum(a, b, c):
        return a + b + c

In [30]:
a = A()
a.sum(1, 2, 3)

6

## staticmethod 구현

- 이 클래스는 하나의 함수를 내부에 저장한다.
- 이 함수를 이름으로 후출하면 내부에 저장된 함수를 전달한다.


In [31]:
class my_staticmethod:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func

In [32]:
class A:
    @my_staticmethod
    def sum(a, b, c):
        return a + b + c

In [33]:
a = A()
a.sum(1, 2, 3)

6